In [2]:
import numpy as np
from scipy.optimize import minimize, LinearConstraint
import scipy.special

In [191]:
beta = 1
B = 0
kappa = 2
# The states are marked mu(x0, k), where x0 is the spin of the root node and k is the number of
# leaves that are spin-up
multiplicity = np.fromfunction(lambda _, k: scipy.special.comb(kappa, k), (2, kappa+1))
eta = multiplicity / (2 ** (kappa+1))
print(f'{eta=}')

def Hamiltonian(x0, k):
    x0 = 2 * x0 - 1
    return beta / 2 * x0 * (2 * k - kappa) + B * x0

def relative_entropy(p, q):
    return np.sum(p * np.log(p / q))

def edge_distribution(p):
    counts = np.vstack((kappa - np.arange(kappa+1),
                        np.arange(kappa+1)))
    return p @ counts.T / kappa

def target_function(mu):
    hamiltonian = np.fromfunction(Hamiltonian, (2, kappa+1))
    exp = np.sum(hamiltonian * mu)
    rel_entr = relative_entropy(mu, eta)
    pi_mu = edge_distribution(mu)
    pi_eta = edge_distribution(eta)
    marg_rel_entr = relative_entropy(pi_mu, pi_eta)
    return exp - rel_entr + marg_rel_entr
    
def norm_constraint(mu):
    return np.sum(mu) - 1

# def norm_constraint_grad(mu):
#     return mu

def admissibility_constraint(mu):
    pi_mu = edge_distribution(mu)
    return pi_mu[0, 1] - pi_mu[1, 0]

# def admissibility_constraint_grad(mu):
#     grad_01 = np.concatenate([kappa - np.arange(kappa+1),
#                               np.zeros((1, kappa+1))], axis=0)
#     grad_10 = np.concatenate([np.zeros((1, kappa+1)),
#                               np.arange(kappa+1)], axis=0)
#     return grad_01 - grad_10

print(f'{edge_distribution(eta)=}')

eta=array([[0.125, 0.25 , 0.125],
       [0.125, 0.25 , 0.125]])
edge_distribution(eta)=array([[0.25, 0.25],
       [0.25, 0.25]])


In [113]:
def optimize(x0, lam0, patience, eps, verbose=True):

    x = x0
    lam = lam0
    
    target = -np.inf
    for it in range(patience):
        if verbose:
            print(f"{it=}")

        # Redefine the function for scipy.optimize.minimize
        # There is probably some fancy functional-programming way to incorporate lambda
        # but you get the point.
        def proximal_target_function(_x):
            _x = _x.reshape((2, kappa+1))
            _mu = np.exp(_x)
            _norm_constraint = norm_constraint(_mu)
            _admissibility_constraint = admissibility_constraint(_mu)
            return -target_function(_mu) \
                - lam[0] * _norm_constraint \
                - lam[1] * _admissibility_constraint \
                + _norm_constraint ** 2 / 2 \
                + 1000 * _admissibility_constraint ** 2 / 2
        
        # Calculate proximal point
        res = minimize(proximal_target_function, x0=x.flatten(), method='l-bfgs-b')
        if not res.success:
            print(f'Proximal Point minimization not successful: {res.message}')
            return x, target
        new_x = res.x.reshape((2, kappa+1))
        
        new_mu = np.exp(new_x.reshape(2, kappa+1))
        new_target = target_function(new_mu)
        _norm_constraint = norm_constraint(new_mu)
        _admissibility_constraint = admissibility_constraint(new_mu)
        if verbose:
            print(f"{new_target=:.4e}\t{_norm_constraint=:.4e}\t{_admissibility_constraint=:.4e}")
            
        # Decide whether to terminate
        if np.abs(target - new_target) < eps and np.all(np.abs(mu - new_mu) < eps) and np.abs(_norm_constraint) < eps and np.abs(_admissibility_constraint) < eps:
            return x, target
        target = new_target
        x = new_x
        mu = new_mu
        
        # Update lambda
        lam[0] = lam[0] - _norm_constraint
        lam[1] = lam[1] - 1000 * _admissibility_constraint
    print(f'Reached Patience: {new_target=:.4e}\t{_norm_constraint=:.4e}\t{_admissibility_constraint=:.4e}')
    return x, target

In [230]:
beta = 1
B = 0

x0 = np.random.uniform(-10, 10, (2, 3))
# x0 = np.zeros((2, kappa+1))

x, target = optimize(x0, lam0=np.zeros(2), patience=2000, eps=1e-6, verbose=False)
mu = np.exp(x)
print(f'{np.sum(mu)=}, {target=}')
mu

np.sum(mu)=0.9999997864538266, target=0.43378066628936596


array([[0.3877483 , 0.10495102, 0.00712468],
       [0.00708234, 0.10503461, 0.38805884]])

### P(X0, X1)

In [207]:
pi_mu = edge_distribution(mu)
print(f'{pi_mu=}')
p_x0 = np.sum(pi_mu, axis=1)
p_x1 = np.sum(pi_mu, axis=0)
print(f'{p_x0=}', f'{p_x1=}')
print(f'{np.outer(p_x0, p_x1)=}')

pi_mu=array([[0.4403988 , 0.05960082],
       [0.05960098, 0.44039769]])
p_x0=array([0.49999962, 0.49999867]) p_x1=array([0.49999978, 0.49999851])
np.outer(p_x0, p_x1)=array([[0.2499997 , 0.24999907],
       [0.24999923, 0.24999859]])


### P(X1, X2)

In [208]:
mu_edge = np.sum(mu, axis=0)
leaf_dist = np.array([[mu_edge[2], mu_edge[1] / 2],
                      [mu_edge[1] / 2, mu_edge[0]],])
print(f'{leaf_dist=}')
p_x1 = np.sum(pi_mu, axis=1)
p_x2 = np.sum(pi_mu, axis=0)
print(f'{p_x1=}', f'{p_x2=}')
print(f'{np.outer(p_x1, p_x2)=}')

leaf_dist=array([[0.39500674, 0.10499177],
       [0.10499177, 0.39500801]])
p_x1=array([0.49999962, 0.49999867]) p_x2=array([0.49999978, 0.49999851])
np.outer(p_x1, p_x2)=array([[0.2499997 , 0.24999907],
       [0.24999923, 0.24999859]])


### P(X1, X2 | X0)

In [220]:
x0 = 1
mu_edge = mu[x0] / np.sum(mu, axis=1)[x0]
print(f'{mu_edge=}')
leaf_dist = np.array([[mu_edge[2], mu_edge[1] / 2],
                      [mu_edge[1] / 2, mu_edge[0]],])
print(f'{leaf_dist=}')
p_x1 = np.sum(pi_mu, axis=1)
p_x2 = np.sum(pi_mu, axis=0)
print(f'{p_x1=}', f'{p_x2=}')
print(f'{np.outer(p_x1, p_x2)=}')

mu_edge=array([0.01420764, 0.20998095, 0.77581142])
leaf_dist=array([[0.77581142, 0.10499047],
       [0.10499047, 0.01420764]])
p_x1=array([0.49999962, 0.49999867]) p_x2=array([0.49999978, 0.49999851])
np.outer(p_x1, p_x2)=array([[0.2499997 , 0.24999907],
       [0.24999923, 0.24999859]])
